# Laboratory Task 4


**Instruction:** Train a linear regression model in PyTorch using using a regression dataset. Use the following parameters:


*   Criterion: MSE Loss
*   Fully Connected Layers x 2
*   Batch Size: 8
*   Optimizer: SGD
*   Epoch: 1000

**Import Libraries**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings('ignore')

**Load the diabetes dataset**

In [61]:
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


**Split into train/test**

In [62]:
X = diabetes.data
y = diabetes.target.reshape(-1, 1)  # reshape y to 2D

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Standardize features and target**

In [63]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

**Convert to PyTorch tensors**

In [64]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

**Create a dataset and dataloader**

In [65]:
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

**Define the model with 2 fully connected layers**

In [66]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = LinearRegressionModel(input_dim=X_tensor.shape[1])

**Loss and optimizer**

In [67]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

**Training loop**

In [68]:
epochs = 1000
for epoch in range(epochs):
    for batch_X, batch_y in loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [100/1000], Loss: 2699.8022
Epoch [200/1000], Loss: 4858.9980
Epoch [300/1000], Loss: 16570.2539
Epoch [400/1000], Loss: 1780.3792
Epoch [500/1000], Loss: 9903.0059
Epoch [600/1000], Loss: 5785.0073
Epoch [700/1000], Loss: 11798.3223
Epoch [800/1000], Loss: 2919.0476
Epoch [900/1000], Loss: 2352.1960
Epoch [1000/1000], Loss: 2710.5901


**Evaluate the model**

In [69]:
with torch.no_grad():
    predictions = model(X_tensor)
    final_loss = criterion(predictions, y_tensor)
    print(f"\nFinal MSE Loss: {final_loss.item():.4f}")


Final MSE Loss: 5930.1606
